<a href="https://colab.research.google.com/github/tuomaseerola/onsetsync/blob/master/onsetsync.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Onsetsync demo as Colab notebook**

A Jupyter notebook with code examples in R for [onsetsync](https://github.com/tuomaseerola/onsetsync) by [Tuomas Eerola](https://www.durham.ac.uk/staff/tuomas-eerola/), [Music and Science Lab]() at [Durham University](https://www.durham.ac.uk) and [Music and Science Lab](https://musicscience.net).

To run the code in your browser, open the file in Colab (click the icon "Open in Colab"). Alternatively, you can download the notebook and run it locally.

File `onsetsync.ipynb` | Version `25/1/2023` | [Back to Index](https://github.com/tuomaseerola/onsetsync/)
---

# Load libraries

In [ ]:
library(ggplot2,quietly = TRUE)
library(tidyverse,quietly = TRUE)

devtools::install_github("tuomaseerola/onsetsync")
library(onsetsync,quietly = TRUE)
ver <- packageVersion('onsetsync')
print(ver)

## Load data (internal)

Here we take a Cuban Son performance titled *Palo Santo* from *IEMP* dataset [@Clayton2020emr] at <https://osf.io/sfxa2/>:

In [ ]:
CSS_Song2 <- onsetsync::CSS_IEMP[[2]]   # Read one song from internal data
CSS_Song2 <- dplyr::select(CSS_Song2,Label.SD,SD,Clave,Bass,Guitar,Tres,
                           Cycle,Isochronous.SD.Time) # Select some columns
print(head(CSS_Song2))

## Load data -- from OSF (optional)

You can read data from OSF or any online repository:

In [ ]:
require(httr)
CSS_Song2_Onset <- get_OSF_csv('8a347') # Get onsets
print(head(CSS_Song2_Onset[,1:8,]))

## Summarise raw data
Count the onsets and give an overall IOIs (difference between onsets within an instrument).

In [ ]:
tab1 <- summarise_onsets(df = CSS_Song2,
                         instr = c('Clave','Bass','Guitar','Tres','Isochronous.SD.Time'))
print(round(tab1,1))

# Visualise

## Visualise onset structures

In [ ]:
fig1 <- plot_by_beat(df = CSS_Song2,
                     instr = c('Bass','Clave','Guitar','Tres'),
                     beat = 'SD',
                     virtual = 'Isochronous.SD.Time',
                     pcols = 2)
options(repr.plot.width=8, repr.plot.height=5)
print(fig1)

## Visualise deviations from isochrony
Show the degree to which the onsets deviate fro isochronous beats.

In [ ]:
fig2 <- plot_by_beat(df = CSS_Song2,
                     instr = c('Bass','Tres'),
                     beat = 'SD',
                     virtual = 'Isochronous.SD.Time',
                     pcols=1,
                     griddeviations = TRUE)
print(fig2)

## Visualise deviations across sub-beat divisions
Maybe a simpler output for one instrument.

In [ ]:
fig3 <- plot_by_beat(df = CSS_Song2,
                     instr = 'Tres',
                     beat = 'SD',
                     virtual = 'Isochronous.SD.Time',
                     pcols=1,
                     griddeviations = TRUE,
                     box = TRUE)
print(fig3)

# Synchrony

## Synchrony between two instruments

To what degree are the pairs of instruments synchronised in this example? Calculation of asynchronies between parts depends on the number of comparable onsets (points at which both are playing on the "same beat". This will be relatively high in music with a homophonic texture, and low in an interlocking or hocketing pattern. In this genre some instruments play on most subdivisions (as with the guitar in this piece) while others are relatively sparse and may only coincide a few times per cycle. In this piece the clave plays on beats `1, 4, 7, 11, 13` while the bass starts off playing on beats `4, 5, 7, 13, 15`; thus they coincide on beats `4`, `7` and `13` on most cycles and these beats are the points at which we can calculate asynchronies. In sum, the number of joint onsets (onset occurring around the same beat) for each pair of instrument varies greatly. In order to keep the mean and standard deviations comparable, we can randomly sample the joint onsets for both instruments.

In [ ]:
d1 <- sync_sample_paired(CSS_Song2, instr1 = "Guitar", instr2 = "Tres", 
  beat = "SD")
dplyr::summarise(data.frame(d1), 
  N = n(), Mean.ms = mean(asynch*1000), Sd.ms = sd(asynch*1000))

## Visualise synchrony between instruments

In [ ]:
inst <- c("Clave","Bass","Guitar","Tres")
dn <- sync_execute_pairs(CSS_Song2, inst, beat = "SD")
print(plot_by_pair(dn))

## Visualise synchrony within sub-divisions of beats

In [ ]:
dn <- sync_execute_pairs(df = CSS_Song2,
  instruments = c("Guitar", "Tres"),
  beat = "SD")
print(knitr::kable(summarise_sync_by_pair(dn, bybeat = TRUE),
  digits = 2,caption = 'Asynchrony between tres and guitar 
  across beat sub-divisions.'))

## Descriptive Statistics


In [ ]:
dl <- sync_sample_paired_relative(df = CSS_Song2, 
  instr = "Bass", instr_ref = c("Guitar","Bass","Tres","Clave"),
  beat = "SD")
print(dl$`Mean pairwise asynchrony`)

dl <- sync_sample_paired_relative(df = CSS_Song2,
  instr = "Clave", instr_ref = c("Guitar","Bass","Tres","Clave"),
  beat = "SD")
print(dl$`Mean pairwise asynchrony`)

## Classic Measures of Synchrony
From Rasch, 1979; Repp & Su, 2013; Wing et al, 2014: Clayton et al., 2020.

In [ ]:
d <- sync_sample_paired(CSS_Song2,"Clave","Bass", beat = "SD")
print(t(summarise_sync(d)))

## Synchrony with other variables (1)
It might be useful to explore whether synchrony is related to tempo or some other aspect of performance. The first explore the cycle duration vs synchrony.

In [ ]:
CSS_Song2 <- CSS_Song2 %>%
  group_by(Cycle) %>%
  mutate(Tempo = 240/(max(Isochronous.SD.Time) - 
                        min(Isochronous.SD.Time))) %>%
  ungroup()

d2 <- sync_sample_paired(CSS_Song2,"Tres","Guitar", 
                         beat = "Tempo")
d3 <- sync_sample_paired(CSS_Song2,"Tres","Guitar", 
                         beat = "Isochronous.SD.Time")
tmp <- data.frame(asynch = d2$asynch*1000, 
                  Tempo = d2$beatL, 
                  Time = d3$beatL)
print(plot_by_var_time(df = tmp,
  var1 = "Time",
  var2 = "asynch",
  var3 = "Tempo",
  ylabel = "Asynchrony (ms)"))

## Synchrony across several performances
Although the number of datasets that are build-in is tiny now, the functions can be used across performances within a corpus format (here `CSS_IEMP` that contains five Cuban Salsa and Son performances).

In [ ]:
corpus <- onsetsync::CSS_IEMP
D <- sync_sample_paired(corpus,"Guitar","Tres", beat="SD")
D <- D$asynch
D$asynch_abs <- abs(D$asynch)*1000
print(plot_by_dataset(D,"asynch_abs","name", box = TRUE))

# Periodicity

## Select extract (a guitar ostinato)
This selects a short excerpt and converts the onsets to a continuous time representations (gaussified onsets).

In [ ]:
extract <- dplyr::filter(CSS_Song2, Guitar >= 60 & Guitar < 65)
fig10 <- gaussify_onsets(extract$Guitar, wlen = 0.2, plot = TRUE)
print(fig10$fig)

## Estimate periodicity
Use autocorrelation, FFT, or smoothed FFT (AKA periodogram in R) to estimate the periodicity. Here the default is periodogram.

In [ ]:
per <- periodicity(extract,instr='Guitar',freq_range=c(0.1,0.6))
print(per$Figure)

## Summarise periodicity
Convenience functions to extract peak of periodicity curve.


In [ ]:
psumm <- summarise_periodicity(per$Curve)
print(paste('Period is:', round(psumm$Per,3), 'seconds'))

You can also convert the peak period into BPM.

In [ ]:
PM <- summarise_periodicity(per$Curve)
print(paste(period_to_BPM(PM$Per)/4,'BPM'))

# Other datasets
There are tens of hours annotated and verified onsets within IEMP datasets [https://osf.io/37fws/](https://osf.io/37fws/). Also other excellent corpora consisting of onsets exists:


  - [CompMusic datasets at UPF: Hindustani (North India), Carnatic (South India), Turkish-makam (Turkey), Arab-Andalusian (Maghreb), and Beijing Opera (China)](https://compmusic.upf.edu/datasets)
  - [Tap & Fiddle: Scandinavian Fiddle Tunes with Accompanying Foot-Tapping](https://zenodo.org/record/4308731)
  - [The MAESTRO (MIDI and Audio Edited for Synchronous Tracks and Organisation)](https://magenta.tensorflow.org/datasets/maestro)
  - [University of Rochester Multi-Modal Musical Performance](http://www2.ece.rochester.edu/projects/air/projects/URMP.html)
  - [Carnatic Music Rhythm Dataset](http://compmusic.upf.edu/carnatic-varnam-dataset)


## Credits
Thanks for **Adrian Poole** and **Simone Tarsitani** for preparing [IEMP Cuban](10.17605/osf.io/s437t) data included here.

## References

Clayton, M., Jakubowski, K., Eerola, T., Keller, P., Camurri, A., Volpe, G., & Alborno, P. (2020). Interpersonal entrainment in music performance: Theory, method and model. _Music Perception, 38(2)_, 136–194.

Clayton, M., Tarsitani, S., Jankowsky, R., Jure, L., Leante, L., Polak, R., Poole, A., Rocamora, M., Alborno, P., Camurri, A., Eerola, T., Jacoby, N., & Jakubowski, K. (2021). The interpersonal entrainment in music performance data collection. _Empirical Musicology Review, 16(1)_, 65–84. [http://dx.doi.org/10.18061/emr.v16i1.7555](http://dx.doi.org/10.18061/emr.v16i1.7555)

Rasch, R. A. (1979). Synchronization in performed ensemble music. _Acta Acustica United with Acustica, 43(2)_, 121–131.

Repp, B. H., & Su, Y.-H. (2013). Sensorimotor synchronization: A review of recent research (2006–2012). _Psychonomic Bulletin & Review, 20(3)_, 403–452.

Wing, A. M., Endo, S., Bradbury, A., & Vorberg, D. (2014). Optimal feedback correction in string quartet synchronization. _Journal of The Royal Society Interface, 11(93)_, 20131125.
